In [1]:
import langchain
print(langchain.__version__)

0.3.16


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

In [11]:
import os

os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "RAG-CHATBOT"
os.environ["GROQ_API_KEY"]="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"


from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [12]:
model.invoke("Hello, tell me a joke")

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 16, 'total_tokens': 57, 'completion_time': 0.034166667, 'prompt_time': 0.00300308, 'queue_time': 0.018053675999999998, 'total_time': 0.037169747}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-de5011b2-9b08-4cf1-95c8-c3af28f33740-0', usage_metadata={'input_tokens': 16, 'output_tokens': 41, 'total_tokens': 57})

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = model | output_parser
result = chain.invoke("Tell me a joke")
print(result)


Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you laugh!


In [8]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
        phone_model: str = Field(description="Name and model of the phone")
        rating: float = Field(description="Overall rating out of 5")
        pros: List[str] = Field(description="List of positive aspects")
        cons: List[str] = Field(description="List of negative aspects")
        summary: str = Field(description="Brief summary of the review")

review_text = """
    Recently upgraded to the iPhone 14, and I'm loving it so far. The display is crystal clear and vibrant,
    definitely an upgrade from my old phone. The new A15 Bionic chip makes everything feel smooth, even heavy apps.
    Face ID is faster and more accurate than ever. Plus, the camera takes amazing photos, especially in low light!
    The battery life has been solid too, getting me through a full day with ease.
    On the downside, the design hasn’t changed much from last year. Also, it’s still pretty expensive, which is a bit of a turnoff.
    And yes, no charger in the box, which feels like a missed opportunity.
    All in all, I’d rate it a 4.5 out of 5. If you’re into Apple’s ecosystem and can justify the price, it’s definitely worth it.
    """

structured_llm = model.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
print(output)
print(output.pros)


phone_model='iPhone 14' rating=4.5 pros=['crystal clear and vibrant display', 'smooth performance', 'fast and accurate Face ID', 'amazing camera', 'solid battery life'] cons=["design hasn't changed much", 'expensive', 'no charger in the box'] summary='A great upgrade from my old phone, but with some drawbacks.'
['crystal clear and vibrant display', 'smooth performance', 'fast and accurate Face ID', 'amazing camera', 'solid battery life']


In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | model | output_parser
result = chain.invoke({"topic": "programming"})
print(result)


Why did the programmer quit his job?

Because he didn't get arrays! (get a raise)


In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
        SystemMessage(content="You are a helpful assistant that tells jokes."),
        HumanMessage(content="Tell me about programming")
    ]
response = model.invoke(messages)
print(response)

template = ChatPromptTemplate([
        ("system", "You are a helpful assistant that tells jokes."),
        ("human", "Tell me about {topic}")
    ])
chain = template | model
response = chain.invoke({"topic": "programming"})
print(response)


content='Programming! It\'s like trying to solve a puzzle blindfolded while being chased by a pack of wild code-sniffing dogs!\n\nBut seriously, programming is like building with Legos, except instead of making a cool castle, you\'re creating a complex piece of software that can do amazing things!\n\nDid you hear about the programmer who quit his job to pursue his dream of becoming a baker? He didn\'t have the "bread" to continue!\n\nOr how about this one: Why do programmers prefer dark mode? Because light attracts bugs!\n\nOkay, okay, I know, I know, programming can be tough and frustrating at times. But don\'t worry, with practice and patience, you\'ll be coding like a pro in no time!\n\nAnd remember, coding is like playing a game – you make mistakes, you learn from them, and you level up!\n\nSo, what kind of programming do you want to learn?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 185, 'prompt_tokens': 28, 'total_tokens': 213, 'completion_time':

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "/"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5]) 


In [ ]:
from langchain_chroma import Chroma

collection_name = "Essays"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


In [ ]:
query = "When was GreenGrow Innovations founded?"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("When was GreenGrow Innovations founded?")
print(retriever_results)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
question = "When was GreenGrow Innovations founded?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | model | StrOutputParser()
print(contextualize_chain.invoke({"input": "Where is it headquartered?", "chat_history": []}))


In [ ]:
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(model, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "When was GreenGrow Innovations founded?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "Where is it headquartered?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")
